In [563]:
from k12libs.utils.nb_easy import k12ai_print
from k12libs.utils.nb_easy import k12ai_get_app_dir

import os
import json
import _jsonnet

import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.core.display import display, Javascript, HTML
from traitlets import Unicode, Bool, validate, TraitError
from ipywidgets import (DOMWidget, register, interact, interactive,  interact_manual,
                        IntSlider, IntText, FloatText, Text, BoundedIntText,
                        BoundedFloatText, Box, HBox, VBox, fixed, Dropdown, Label,
                        Layout, Tab, Accordion, ToggleButton, ToggleButtons, Checkbox)

In [564]:
cv_schema_dir = os.path.join(k12ai_get_app_dir('cv'), 'templates', 'schema')
cv_json = _jsonnet.evaluate_file(os.path.join(cv_schema_dir, 'k12cv.jsonnet'))
k12ai_print(cv_json)

{
    "description": "k12cv configure test\n",
    "objs": [
        {
            "name": {
                "cn": "data",
                "en": "data"
            },
            "objs": [
                {
                    "name": {
                        "cn": "Dataset",
                        "en": "Dataset"
                    },
                    "objs": [],
                    "type": "accordion"
                },
                {
                    "name": {
                        "cn": "Transform",
                        "en": "Transform"
                    },
                    "objs": [
                        {
                            "name": {
                                "cn": "Phase",
                                "en": "Phase"
                            },
                            "objs": [
                                {
                                    "name": {
                                        "cn": "train",
                     

In [565]:
k12cv_conf = json.loads(cv_json)

In [566]:
out = widgets.Output()

def k12widget(method):
    def _widget(self, *args, **kwargs):
        root, observes, cb = method(self, *args, **kwargs)
        def _on_value_change(change, cb):
            with out:
                clear_output()
                print(change)
            if cb:
                cb(change)
                
        for per in observes:
            per.observe(lambda change, cb = cb: _on_value_change(change, cb), 'value')
        return root
    return _widget

class K12Context():
    def __init__(self, defaults=None):
        self.wids_map = {}
        self.defaults = defaults
        self.vlo = Layout(
            width='auto',
            align_items='stretch',
            justify_content='flex-start',
            margin='10px 10px 10px 10px',
            # border='solid 1px red',
        )
        self.hlo = Layout(
            width='100%',
            flex_flow='row wrap',
            align_items='stretch',
            justify_content='flex-start',
            # border='solid 1px blue',
        )

    @k12widget
    def Bool(self, wid, *args, **kwargs):
        self.wids_map[wid] = Checkbox(*args, **kwargs)
        def _value_change(change):
            print("Checkbox here")
        return self.wids_map[wid], [self.wids_map[wid]], _value_change
    
    @k12widget
    def Int(self, wid, *args, **kwargs):
        self.wids_map[wid] = BoundedIntText(*args, **kwargs)
        def _value_change(change):
            print("BoundedIntText here")
        return self.wids_map[wid], [self.wids_map[wid]], _value_change
    
    @k12widget
    def Float(self, wid, *args, **kwargs):
        self.wids_map[wid] = BoundedFloatText(*args, **kwargs)
        def _value_change(change):
            print("BoundedFloatText here")
        return self.wids_map[wid], [self.wids_map[wid]], _value_change
    
    @k12widget
    def String(self, wid, *args, **kwargs):
        self.wids_map[wid] = Text(*args, **kwargs)
        return self.wids_map[wid], [self.wids_map[wid]], None
    
    @k12widget
    def IntArray(self, wid, *args, **kwargs):
        self.wids_map[wid] = Text(*args, **kwargs)
        def _value_change(change):
            print("IntArray here")
            print(json.loads(change['new']))
        return self.wids_map[wid], [self.wids_map[wid]], _value_change
    
    @k12widget
    def StringEnum(self, wid, *args, **kwargs):
        self.wids_map[wid] = Dropdown(*args, **kwargs)
        def _value_change(change):
            print("StringEnum here")
        return self.wids_map[wid], [self.wids_map[wid]], _value_change
    
    @k12widget
    def BoolTrigger(self, wid, *args, **kwargs):
        parent_box = VBox(layout = self.hlo)
        parent_box.trigger_box = VBox(layout = self.hlo)
        wdg = Checkbox(*args, **kwargs)
        wdg.parent_box = parent_box
        def _switch_widget(wdg, value):
            if value:
                wdg.parent_box.children = [wdg, wdg.parent_box.trigger_box]
            else:
                wdg.parent_box.children = [wdg]
                
        def _value_change(change):
            print("BoolTrigger here")
            wdg = change['owner']
            val = change['new']
            _switch_widget(wdg, val)
        _switch_widget(wdg, wdg.value)
        return parent_box, [wdg], _value_change
    
    @k12widget
    def StringEnumTrigger(self, wid, *args, **kwargs):
        parent_box = VBox(layout = self.hlo)
        wdg = Dropdown(*args, **kwargs)
        wdg.parent_box = parent_box
        parent_box.trigger_box = {
         value: VBox(layout = self.hlo) for _, value in wdg.options
        }

        def _switch_widget(wdg, value):
            wdg.parent_box.children = [wdg, wdg.parent_box.trigger_box[value]]
            
        def _value_change(change):
            print("StringEnumTrigger here")
            wdg = change['owner']
            val = change['new']
            _switch_widget(wdg, val)
        _switch_widget(wdg, wdg.value)
        return parent_box, [wdg], _value_change
    
    @k12widget
    def StringEnumArrayTrigger(self, wid, options):
        root_vbox = VBox(layout = self.vlo)
        options_hbox = HBox(layout = self.hlo)
        dynamic_hbox = HBox(layout = self.hlo)
        observes = []
        for name, real in options:
            btn = ToggleButton(description=name)
            btn.parent_box = dynamic_hbox
            btn.trigger_box = VBox(layout = self.vlo)
            observes.append(btn)
        options_hbox.children = options_hbox
        root_vbox.children = [options_hbox, dynamic_hbox]
        
        def _switch_widget(wdg, value):
            if value:
                wdg.parent_box.children = wdg.parent_box.children + [wdg.trigger_box]
            else:
                wdg.parent_box.children = [wdg for wdg in wdg.parent_box.children if wdg != wdg.trigger_box]
        
        def _value_change(change):
            print("StringEnumArrayTrigger here")
            wdg = change['owner']
            val = change['new']
            _switch_widget(wdg, val)
            
        return root_vbox, observes, _value_change
    
    @k12widget
    def StringEnumGroupTrigger(self, wid, options, groups):
        parent_box = VBox(layout = self.vlo)
        parent_box.trigger_box = {}
        options_hbox = HBox(layout = self.hlo)
        dynamic_hbox = HBox(layout = self.hlo)
        observes = []
        for name, real in options:
            dpd = Dropdown(options=groups,
                           description=name,
                           style = {'description_width': 'initial'})
            dpd.name = real
            dpd.wid = wid + '.' + real 
            dpd.parent_box = parent_box
            dpd.parent_box.trigger_box[real] = VBox(layout = self.vlo)
            observes.append(dpd)            
        options_hbox.children = tuple(observes)
        parent_box.children = (options_hbox, dynamic_hbox)
        def _value_change(change):
            print("StringEnumGroupTrigger here")
            wdg = change['owner']
            val = change['new']
            parent_box = wdg.parent_box
            dynamic_box = parent_box.children[1]
            trigger_box = parent_box.trigger_box[wdg.name]
            if val == 'none':
                if trigger_box in dynamic_box.children:
                    dynamic_box.children = [ w for w in dynamic_box.children if id(w) != id(trigger_box)]
            else:
                if trigger_box not in dynamic_box.children:
                    trigger_box.children = [Label(wdg.name)] + list(trigger_box.children)
                    dynamic_box.children = list(dynamic_box.children) + [trigger_box]
        return parent_box, observes, _value_change
    
    @k12widget
    def ObjectTrigger(self, wid, *args, **kwargs):
        wdg = ToggleButtons(*args, **kwargs)
        wdg.parent_box = VBox(layout = self.hlo)
        wdg.parent_box.trigger_box = {}
        for name in wdg.options:
            wdg.parent_box.trigger_box[name] = VBox(layout = self.hlo)

        def _switch_widget(wdg, value):
            parent = wdg.parent_box
            parent.children = [wdg, parent.trigger_box[value]]
            
        def _value_change(change):
            print("ObjectEnumTrigger here")
            wdg = change['owner']
            val = change['new']
            _switch_widget(wdg, val)
        _switch_widget(wdg, wdg.value)
        return wdg.parent_box, [wdg], _value_change
        
k12Context = K12Context()

In [568]:
lan = 'en'
box_layout = Layout(display='flex',
                flex_flow='row wrap',
                align_items='stretch',
                justify_content='flex-start',
                width='100%')

vlo = Layout(width='100%')
hlo = Layout(flex_flow='row wrap', width='100%')

_basic_types = ['int', 'float', 'bool', 'string', 'string-enum']

def _parse_config(widget, config):
    __id_ = config.get('_id_', None) or ''
    _name = config.get('name', None)
    _type = config.get('type', None)
    _objs = config.get('objs', None) or {}
    if _type == 'page':
        wdg = Tab(layout=Layout(width='100%'))
        widget.children = list(widget.children) + [wdg]
        for obj in _objs:
            _parse_config(wdg, obj)
        return widget
    elif _type == 'tab':
        widget.set_title(len(widget.children), _name[lan])
        wdg = VBox(layout = box_layout)
        widget.children = list(widget.children) + [wdg] 
        for obj in _objs:
            _parse_config(wdg, obj)
        return widget
    elif _type == 'accordion':
        for wdt in widget.children:
            if isinstance(wdt, Accordion):
                accord = wdt
                break
        else:
            accord = Accordion(layout=Layout(
                display='flex',
                align_items='stretch',
                justify_content='flex-start',
                width='100%'
            ))
            widget.children = list(widget.children) + [accord]
        accord.set_title(len(accord.children), _name[lan])
        basic_box = HBox(layout = hlo)
        compx_box = HBox(layout = vlo)
        nxtwdt = VBox((basic_box, compx_box), layout = box_layout)
        accord.children = list(accord.children) + [nxtwdt]
        for obj in _objs:
            typ = obj.get('type', None)
            if typ in _basic_types:
                _parse_config(basic_box, obj)
            else:
                _parse_config(compx_box, obj)
        return
    elif _type == 'bool':
        default = config.get('default', False)
        wdg = k12Context.Bool(__id_,
             description = _name[lan],
             value = default,
         )
        widget.children = list(widget.children) + [wdg]
        return widget
    elif _type == 'int':
        default = config.get('default', 0)
        wdg = k12Context.Int(__id_,
            description = _name[lan],
            value = default,
            min = 0,
            max = 100000,
            step = 1,
        )
        widget.children = list(widget.children) + [wdg]
        return widget
    elif _type == 'float':  # leaf node
        default = config.get('default', 0.0)
        wdg = k12Context.Float(__id_,
            description = _name[lan],
            value = default,
            min = 0.0,
            max = 100.0,
            step = 0.1,
        )
        widget.children = list(widget.children) + [wdg]
        return widget
    elif _type == 'string':
        default = config.get('default', 'none')
        wdg = k12Context.String(__id_,
            description = _name[lan],
            value = default,
        )
        widget.children = list(widget.children) + [wdg]
        return widget
    elif _type == 'int-array':
        default = config.get('default', '[]')
        wdg = k12Context.IntArray(__id_,
            description = _name[lan],
            value = json.dumps(default),
        )
        widget.children = list(widget.children) + [wdg]
        return widget
    elif _type == 'float-array':
        print('not impl')
        return
    elif _type == 'string-enum': # leaf node
        default = config.get('default', 'None')
        options = []
        if _objs and len(_objs) > 0:
            for obj in _objs:
                options.append((obj['name'][lan], obj['value']))
            wdg = k12Context.StringEnum(__id_,
                options = options,
                value = default,
                description = _name[lan])
            widget.children = list(widget.children) + [wdg]
        return widget
    elif _type == 'bool-trigger':
        # widget is compx_box
        default = config.get('default', False)
        if _objs and len(_objs) > 0:
            wdg = k12Context.BoolTrigger(__id_,
                  value = default,
                  description = _name[lan])
            print(_objs)
            for obj in _objs:
                print("#######1")
                print(obj)
                print("#######2")
                _parse_config(wdg.trigger_box, obj['trigger'])
            widget.children = list(widget.children) + [wdg]
        return widget
    elif _type == 'string-enum-trigger':
        default = config.get('default', 'none')
        options = []
        if _objs and len(_objs) > 0:
            for obj in _objs:
                options.append((obj['name'][lan], obj['value']))
            wdg = k12Context.StringEnumTrigger(__id_,
                options = options,
                value = default,
                description = _name[lan])
            for obj in _objs:
                print(obj['trigger'])
                _parse_config(wdg.trigger_box[obj['value']], obj['trigger'])                    
            widget.children = list(widget.children) + [wdg]        
        return widget
    elif _type == 'string-enum-group-trigger':
        options = []
        groups = []
        for obj in _objs:
            options.append((obj['name'][lan], obj['value']))
        for grp in config.get('groups'):
            groups.append((grp['name'][lan], grp['value']))
        wdg = k12Context.StringEnumGroupTrigger(
            __id_,
            options,
            groups
        )
        for obj in _objs:
            _parse_config(wdg.trigger_box[obj['value']], obj['trigger'])                    

        widget.children = list(widget.children) + [wdg]        
        return
    elif _type == 'string-enum-array-trigger':
        default = config.get('default', '[]')
        options = []
        for obj in _objs:
            options.append((obj['name'][lan], obj['value']))
        wdg = k12Context.StringEnumArrayTrigger(__id_,
            options =options)
        widget.children = list(widget.children) + [wdg.parent_box]        
        return
    elif _type == 'object-trigger':
        default = config.get('default', 'none')
        options = []
        for obj in _objs:
            options.append(obj['name'][lan])
        wdg = k12Context.ObjectTrigger(
            __id_,
            options = options,
            description = _name[lan])
        for obj in _objs:
            _parse_config(wdg.trigger_box[obj['name'][lan]], obj)                    
        widget.children = list(widget.children) + [wdg]        
        return widget
    elif _type == 'object':
        for obj in _objs:
            _parse_config(widget, obj)
        return widget
    else:
        print('unkown', _name)
        return 

page = Box()
_parse_config(page, k12cv_conf)
display(page, out)

[{'name': {'cn': 'True', 'en': 'True'}, 'trigger': {'objs': [{'_id_': 'lr.warm_iters', 'default': 1000, 'name': {'cn': 'warm iter', 'en': 'warm iter'}, 'type': 'int'}, {'_id_': 'lr.warm.power', 'default': 1, 'name': {'cn': 'Power', 'en': 'Power'}, 'type': 'float'}, {'_id_': 'lr.warm.freeze_backbone', 'default': False, 'name': {'cn': 'freeze backbone', 'en': 'freeze backbone'}, 'type': 'bool'}], 'type': 'object'}, 'value': True}, {'name': {'cn': 'False', 'en': 'False'}, 'trigger': {}, 'value': False}]
#######1
{'name': {'cn': 'True', 'en': 'True'}, 'trigger': {'objs': [{'_id_': 'lr.warm_iters', 'default': 1000, 'name': {'cn': 'warm iter', 'en': 'warm iter'}, 'type': 'int'}, {'_id_': 'lr.warm.power', 'default': 1, 'name': {'cn': 'Power', 'en': 'Power'}, 'type': 'float'}, {'_id_': 'lr.warm.freeze_backbone', 'default': False, 'name': {'cn': 'freeze backbone', 'en': 'freeze backbone'}, 'type': 'bool'}], 'type': 'object'}, 'value': True}
#######2
#######1
{'name': {'cn': 'False', 'en': 'Fals

AttributeError: 'str' object has no attribute 'get'